In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Load and combine datasets
data = pd.read_csv('client_1.csv')

# Load data
X = data.drop(columns=['TenYearCHD'])
y = data['TenYearCHD']



# Split data (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.long)

# Create DataLoaders
batch_size = 128
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)

# Define the MLP Model
class MLP(nn.Module):
    def __init__(self, input_size, hidden1, hidden2, hidden3, hidden4, output_size, dropout_prob=0.2):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1)
        self.bn1 = nn.BatchNorm1d(hidden1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)

        self.fc2 = nn.Linear(hidden1, hidden2)
        self.bn2 = nn.BatchNorm1d(hidden2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)

        self.fc3 = nn.Linear(hidden2, hidden3)
        self.bn3 = nn.BatchNorm1d(hidden3)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_prob)

        self.fc4 = nn.Linear(hidden3, hidden4)  # NEW LAYER
        self.bn4 = nn.BatchNorm1d(hidden4)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_prob)

        self.fc5 = nn.Linear(hidden4, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.dropout3(x)

        x = self.fc4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.dropout4(x)

        x = self.fc5(x)
        return x

# Define new hyperparameters
input_size = X_train.shape[1]
hidden1, hidden2, hidden3, hidden4 = 512, 512, 256, 128  # Added new layer
output_size = 2
learning_rate = 0.0008
epochs = 300

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(input_size, hidden1, hidden2, hidden3, hidden4, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)  # L2 regularization

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.CyclicLR(
    optimizer, 
    base_lr=0.0001,  
    max_lr=0.001,  
    step_size_up=len(train_loader) * 10,  # Update over 10 epochs
    mode='triangular'
)

scheduler2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=20)

# Training loop with validation accuracy tracking
best_val_acc = 0.0

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss += loss.item()

    # Validation step
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total  # Compute validation accuracy
    scheduler2.step(val_acc)  # Update ReduceLROnPlateau

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")  # Save best model

    # Print progress every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(train_loader):.4f}, LR: {scheduler.get_last_lr()[0]:.6f}, Val Accuracy: {val_acc:.4f}')

# Load the best model for final evaluation
model.load_state_dict(torch.load("best_model.pth"))
print(f'Best Validation Accuracy: {best_val_acc:.4f}')


Epoch [10/300], Loss: 0.5964, LR: 0.001000, Val Accuracy: 0.6917
Epoch [20/300], Loss: 0.5141, LR: 0.000100, Val Accuracy: 0.6979
Epoch [30/300], Loss: 0.5181, LR: 0.001000, Val Accuracy: 0.7208
Epoch [40/300], Loss: 0.4423, LR: 0.000100, Val Accuracy: 0.7083
Epoch [50/300], Loss: 0.4688, LR: 0.001000, Val Accuracy: 0.7188
Epoch [60/300], Loss: 0.3714, LR: 0.000100, Val Accuracy: 0.7354
Epoch [70/300], Loss: 0.4283, LR: 0.001000, Val Accuracy: 0.7771
Epoch [80/300], Loss: 0.3164, LR: 0.000100, Val Accuracy: 0.7708
Epoch [90/300], Loss: 0.3724, LR: 0.001000, Val Accuracy: 0.7646
Epoch [100/300], Loss: 0.2295, LR: 0.000100, Val Accuracy: 0.7708
Epoch [110/300], Loss: 0.2976, LR: 0.001000, Val Accuracy: 0.7625
Epoch [120/300], Loss: 0.2136, LR: 0.000100, Val Accuracy: 0.7854
Epoch [130/300], Loss: 0.2757, LR: 0.001000, Val Accuracy: 0.7646
Epoch [140/300], Loss: 0.1715, LR: 0.000100, Val Accuracy: 0.7917
Epoch [150/300], Loss: 0.2474, LR: 0.001000, Val Accuracy: 0.7750
Epoch [160/300], Lo

C:\Users\Ashan\AppData\Local\Temp\ipykernel_13888\4156566384.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))
